In [16]:
from config import PROJECT_ID, LOCATION, MEMORY_ENGINE_DISPLAY_NAME, INPUT_FILE_PATH,MAIN_AGENT,CRITIC_MODEL,CODE_MODEL, API_TYPE_GEMINI,API_TYPE_SONNET,QUICK_SMART_MODEL,ANTHROPIC_API_KEY,basic_config_agent
from dynamic_graph.outputsModels import *
from dynamic_graph.main_graph_functions import run_collaborative_planning
from prompts import PromptEngine

In [13]:
# --- Konfiguracja czatu grupowego ---
main_agent_configuration={"cache_seed": 42,"seed": 42,"temperature": 0.0,
                        "config_list": basic_config_agent(agent_name=MAIN_AGENT, api_type=API_TYPE_GEMINI, location=LOCATION, project_id=PROJECT_ID)}
critic_agent_configuration ={"cache_seed": 42,"seed": 42,"temperature": 0.0,
                        "config_list": basic_config_agent(api_key=ANTHROPIC_API_KEY,agent_name=CRITIC_MODEL, api_type=API_TYPE_SONNET)}


In [14]:
if __name__ == '__main__':
    # Konfiguracja dla lekkiego i szybkiego modelu Rutera
    router_llm_config = basic_config_agent(agent_name=QUICK_SMART_MODEL, api_type=API_TYPE_GEMINI)
    
    # Wysokopoziomowa misja od użytkownika
    mission = "Zbudować i zweryfikować podstawowy model przyczynowy na podstawie załadowanych danych."
    
    # Uruchom całą fazę planowania jedną komendą
    final_plan = run_collaborative_planning(mission, router_llm_config)
    
    if final_plan:
        print("\n" + "="*50)
        print("### OTRZYMANY PLAN GOTOWY DO WYKONANIA ###")
        print(final_plan.model_dump_json(indent=2, by_alias=True))
        
        # W kolejnym kroku, ten obiekt `final_plan` zostałby przekazany
        # do "Fabryki Grafów", aby zbudować i uruchomić przepływ.
        # build_and_run_graph(final_plan, ...)
    else:
        print("\nNie udało się wypracować planu. Sprawdź logi w `reports/`.")

    
    


### ROZPOCZYNANIE FAZY PROJEKTOWEJ ###
--- RUTER AGENTÓW: Uruchamiam uniwersalny wybór zespołu... ---


NameError: name 'PromptEngine' is not defined